# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [60]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [61]:
adult_dt.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [62]:
from sklearn.model_selection import train_test_split

X = adult_dt.drop(columns='income')  
Y = adult_dt['income']              

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

print(f"Training set size: X_train: {X_train.shape}, Y_train: {Y_train.shape}")
print(f"Testing set size: X_test: {X_test.shape}, Y_test: {Y_test.shape}")

Training set size: X_train: (22792, 14), Y_train: (22792,)
Testing set size: X_test: (9769, 14), Y_test: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import set_config

set_config(display='diagram')

numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

num_transforms = Pipeline(steps=[
    ('KNNImputer', KNNImputer(n_neighbors=7, weights='distance')), 
    ('RobustScaler', RobustScaler())   
])

cat_transforms = Pipeline(steps=[
    ('SimpleImputer', SimpleImputer(strategy='most_frequent')),  
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))  
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, numerical_features),
        ('cat_transforms', cat_transforms, categorical_features)
    ]
)

preprocessed_data = preprocessor.fit_transform(adult_dt)
print(f"Shape of preprocessed data: {preprocessed_data.shape}")
preprocessor

Shape of preprocessed data: (32561, 107)


ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('KNNImputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('RobustScaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('SimpleImputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('random_forest', RandomForestClassifier(random_state=42))  
])

model_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('KNNImputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('random_forest', RandomForestClassifier(random_state=42))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [65]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
import pandas as pd

def define_scoring_metrics():
    return {
        'neg_log_loss': make_scorer(log_loss, needs_proba=True),       
        'roc_auc': make_scorer(roc_auc_score, needs_proba=True),        
        'accuracy': make_scorer(accuracy_score),                      
        'balanced_accuracy': make_scorer(balanced_accuracy_score)       
    }

def evaluate_model(model, X, y, scoring, cv_folds=5):
    # 交叉验证
    cv_results = cross_validate(
        model, X, y, 
        cv=cv_folds, 
        scoring=scoring, 
        return_train_score=True
    )

    # 输出结果
    results_df = pd.DataFrame({
        'Metric': [metric.replace('_', ' ').title() for metric in scoring.keys()],
        'Train Score': [cv_results[f'train_{metric}'].mean() for metric in scoring.keys()],
        'Validation Score': [cv_results[f'test_{metric}'].mean() for metric in scoring.keys()]
    })
    
    print("Training and Validation Results:")
    print(results_df.to_string(index=False))  

    return cv_results

scoring_metrics = define_scoring_metrics()
cv_results = evaluate_model(model_pipeline, adult_dt, Y, scoring=scoring_metrics)

C:\Users\aaron\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
C:\Users\aaron\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\aaron\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training and Validation Results:
           Metric  Train Score  Validation Score
     Neg Log Loss     0.080596          0.375134
          Roc Auc     1.000000          0.902323
         Accuracy     0.999969          0.854704
Balanced Accuracy     0.999969          0.775385


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [66]:
def create_and_sort_results_dataframe(cv_results, sort_by='test_neg_log_loss'):
    results_df = pd.DataFrame({
        'fold': range(1, len(cv_results['train_neg_log_loss']) + 1),  
        'train_neg_log_loss': cv_results['train_neg_log_loss'],
        'test_neg_log_loss': cv_results['test_neg_log_loss'],
        'train_roc_auc': cv_results['train_roc_auc'],
        'test_roc_auc': cv_results['test_roc_auc'],
        'train_accuracy': cv_results['train_accuracy'],
        'test_accuracy': cv_results['test_accuracy'],
        'train_balanced_accuracy': cv_results['train_balanced_accuracy'],
        'test_balanced_accuracy': cv_results['test_balanced_accuracy']
    })
    return results_df.sort_values(by=sort_by, ascending=True)

sorted_results_df = create_and_sort_results_dataframe(cv_results)

print(sorted_results_df)

   fold  train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
3     4            0.080591           0.356018            1.0      0.903304   
2     3            0.080695           0.369435            1.0      0.902365   
4     5            0.081341           0.375717            1.0      0.906950   
0     1            0.080468           0.379042            1.0      0.902153   
1     2            0.079882           0.395460            1.0      0.896845   

   train_accuracy  test_accuracy  train_balanced_accuracy  \
3        1.000000       0.858108                 1.000000   
2        0.999962       0.852119                 0.999975   
4        0.999962       0.857647                 0.999975   
0        0.999962       0.853677                 0.999920   
1        0.999962       0.851966                 0.999975   

   test_balanced_accuracy  
3                0.776560  
2                0.778277  
4                0.782353  
0                0.770268  
1                0

Calculate the mean of each metric. 

In [67]:
def calculate_mean_metrics(results_df):
    return {
        'Mean Train Negative Log Loss': -results_df['train_neg_log_loss'].mean(),  
        'Mean Test Negative Log Loss': -results_df['test_neg_log_loss'].mean(),
        'Mean Train ROC AUC': results_df['train_roc_auc'].mean(),
        'Mean Test ROC AUC': results_df['test_roc_auc'].mean(),
        'Mean Train Accuracy': results_df['train_accuracy'].mean(),
        'Mean Test Accuracy': results_df['test_accuracy'].mean(),
        'Mean Train Balanced Accuracy': results_df['train_balanced_accuracy'].mean(),
        'Mean Test Balanced Accuracy': results_df['test_balanced_accuracy'].mean()
    }

def display_mean_metrics(mean_metrics):
    mean_metrics_df = pd.DataFrame([mean_metrics])
    print("Mean Performance Metrics:")
    print(mean_metrics_df.to_string(index=False))

mean_metrics = calculate_mean_metrics(results_df)
display_mean_metrics(mean_metrics)

Mean Performance Metrics:
 Mean Train Negative Log Loss  Mean Test Negative Log Loss  Mean Train ROC AUC  Mean Test ROC AUC  Mean Train Accuracy  Mean Test Accuracy  Mean Train Balanced Accuracy  Mean Test Balanced Accuracy
                    -0.080596                    -0.375134            0.999969           0.775385             0.999969            0.854704                      0.999969                     0.775385


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

def split_data(X, y, test_size=0.3, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

def fit_model(model, X_train, y_train):
    model.fit(X_train, y_train)

def evaluate_performance(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)
    
    metrics = {
        'Negative Log Loss': log_loss(y_test, y_pred_proba),
        'ROC AUC': roc_auc_score(y_test, y_pred_proba[:, 1]),
        'Accuracy': accuracy_score(y_test, y_pred),
        'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred)
    }
    
    return metrics

def display_metrics(metrics):
    print("Performance Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

X_train, X_test, Y_train, Y_test = split_data(adult_dt, Y)
fit_model(model_pipeline, X_train, Y_train)
performance_metrics = evaluate_performance(model_pipeline, X_test, Y_test)
display_metrics(performance_metrics)

Performance Metrics:
Negative Log Loss: 0.3973
ROC AUC: 0.9000
Accuracy: 0.8552
Balanced Accuracy: 0.7752


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)
Recoding the income variable right at the start makes things a lot easier! By immediately converting it to a binary format—where 1 means >50K and 0 means <=50K—we ensure that the data is consistent and ready for modeling without needing extra steps later on. This setup also works perfectly with scikit-learn's requirements for metrics like ROC AUC, which assume that the positive class is labeled as 1. Doing it this way upfront saves time and helps avoid mistakes down the line, especially when we’re calculating performance metrics that rely on this standard encoding.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.